In [5]:
import pandas as pd
import numpy as np
import pickle
import re
import timeit
import spacy
import copy

import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel, HdpModel, LdaModel, LdaMulticore
from nltk.corpus import stopwords
import helper as he
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style("whitegrid") 

with open('../data/preprocessed_data/doc_indexes/gst.pkl','rb') as f:
    texts,INITIAL_DOC_SIZE, DOC_TEMPORAL_INCREMENT = pickle.load(f)

with open('../data/preprocessed_data/corpus_dict/gst_corp.pkl', 'rb') as f:
    data_lemmatized, _, _ = pickle.load(f)

ModuleNotFoundError: No module named 'seaborn'

In [4]:
print('Building Dictionary')
#Dictionary is built over entire set of documents
id2word = Dictionary(documents=data_lemmatized)

Building Dictionary


NameError: name 'data_lemmatized' is not defined

In [ ]:
#Golden Standard Model
print('Building Golden Corpus')
golden_corpus = [id2word.doc2bow(doc) for doc in data_lemmatized]
golden_lda = LdaMulticore(golden_corpus, num_topics=35, id2word=id2word,
                   workers=3, chunksize=2000, passes=10, batch=False)
print('Finished Building Golden Corpus')

In [ ]:
def get_positives_arr(initial_doc_size, doc_increment):
    
    # Set Data State to that of existing model in simulation
    data = data_lemmatized[:initial_doc_size]
    corpus = [id2word.doc2bow(doc) for doc in data]

    print('Building Initial Model')
    # Building for the first time - To be considered as the starting/existing model in simulation.
    running_lda = LdaMulticore(corpus, num_topics=35, id2word=id2word,
                       workers=3, chunksize=2000, passes=10, batch=False)
    print('Finished Building Initial Model')
    
    
    count = initial_doc_size
    positive_arr=[]
    no=0
    total_len=len(data_lemmatized)
    for i in doc_increment:
        no+=1
        new_docs = data_lemmatized[count:count+i]
        count+=i
        print('Progress upto Document no.',count,'/',total_len)
        print('No. of New Docs:',i)

        new_corp = [id2word.doc2bow(doc) for doc in new_docs]

        positive_arr.append(calc_confusion_matrix(golden_lda, running_lda, new_corp))
        
        if(i!=doc_increment[-1]):
            print('MODEL NO:'+str(no))
            running_lda.update(new_corp)
            print('MODEL DONE')
    return positive_arr

In [ ]:
def get_doc_parameters(initial,k):
    arr = []
    arr.append(INITIAL_DOC_SIZE)
    arr.extend(DOC_TEMPORAL_INCREMENT)
    for i in range(1,len(arr)):
        arr[i]=arr[i]+arr[i-1]
        
    doc_sizes = []
    count=1
    for i in range(initial+1,len(arr)):
        if(count%k==0):
            doc_sizes.append(arr[i])
            count=1
        count+=1
        
    doc_intervals = [arr[initial],doc_sizes[0]-arr[initial]]
    for i in range(1,len(doc_sizes)):
        doc_intervals.append(doc_sizes[i]-doc_sizes[i-1])
    
    return doc_intervals

In [ ]:
def add_row_to_df(df,positive_arr,k):
    length = len(df)
    count=length
    for i in positive_arr:
        df.loc[count]=[i,count-length,k]
        count+=1
    return df

In [ ]:
def calc_confusion_matrix(model1, model2, corpus1, doc_max=True):
    lda_corpus_1 = [max(prob, key=lambda y:y[1])
                    for prob in model1[corpus1]]
    lda_corpus_2 = [max(prob, key=lambda y:y[1])
                    for prob in model2[corpus1]]
    positive = 0
    negative = 0
    upper_limit = len(lda_corpus_1)
    total_permutations = upper_limit * \
        (upper_limit-1)/2  # nC2 combinations
    for i in range(upper_limit):
        for j in range(i+1, upper_limit):
            if(lda_corpus_1[i][0] == lda_corpus_1[j][0] and lda_corpus_2[i][0] == lda_corpus_2[j][0]):
                positive = positive+1
            elif(lda_corpus_1[i][0] != lda_corpus_1[j][0] and lda_corpus_2[i][0] == lda_corpus_2[j][0]):
                negative = negative+1
            elif(lda_corpus_1[i][0] == lda_corpus_1[j][0] and lda_corpus_2[i][0] != lda_corpus_2[j][0]):
                negative = negative+1
            elif(lda_corpus_1[i][0] != lda_corpus_1[j][0] and lda_corpus_2[i][0] != lda_corpus_2[j][0]):
                positive = positive+1
    answers_positive = (round(positive*100/total_permutations, 2))
    return answers_positive

In [ ]:
f = open('temp.pkl','wb')
for i in range(100,101):
# for i in range(2,3):
    df = pd.DataFrame(columns=['positive_value','index','k'])
    for j in range(1,2):
#     for j in range(32,34):
        print('Loop Parameters-',(i,j))
        doc_params = get_doc_parameters(i,j)
        positive = get_positives_arr(doc_params[0],doc_params[1:])
        pickle.dump(((i,j),positive),f)
        df = add_row_to_df(df,positive,j)
        
        fig, ax = plt.subplots()
        sns_plot = sns.violinplot(x = 'k', y = 'positive_value', data = df)
        sns_plot = sns.pointplot(x='k', y='positive_value', data=df.groupby('k', as_index=False).median(), ax=ax, color='k', linestyle='--')

        plt.savefig("./temp/initial_"+str(i)+".eps")
        
        print('-----------------------------------------------')
    df.to_csv(r'./temp/inital_'+str(i)+'.csv')
f.close()

Model built on 1 month data <br>
Incremental updates done on 30*2 weeks data